## 📘 Kernel Density Estimation

Any set of $M$ points ${x_1, x_2, ..., x_M}$ each of dimensionality $N$ follow some probability distribution $P(x)$. In many cases, the form of $P(x)$ is known and we only need to estimate its parameters (e.g., normal distribution with mean $\mu$ and covariance $Σ$); however, in the general case the form of the distribution is unknown. Kernel density estimation helps us estimate $P(x)$ without assuming any form for it and while only using the available points ${x_1, x_2, ..., x_M}$.

### ✦ Given
Set of $M$ points ${x_1, x_2, ..., x_M}$ each of dimensionality $N$

### ✦ Wanted 
An accurate estimate of the probability density function $P(x)$

### ✦ Operation

1. Choose a bump function $g(x)$ that integrates to $1$
2. Choose a bandwidth $h$ that will control the spread of the bump function
3. Estimate $P(x)$ as follows:
Define
$$ϕ(x)=\frac{1}{h}g(\frac{x}{h})   \tag{1}$$ 
Then compute
$$P(x) = \frac{1}{M} \sum_{m=1}^{M} \phi(x-x_m) \tag{2}$$


**You are required to implement seven methods in `KDEstimator.py` where using any for loop is not allowed:**
<br><br>
```python
__init__(self, bump='Gauss', bandwidth='Silverman')
```
- Sets the chosen bump function and bandwidth which can be that of `Silverman` or `Scott` (introduced below) or any positive real number
<br><br>
```python
fit(self, x_train)
```
- Computes the bandwidth and stores the training data and its shape to be used later for the computation of $P(x)$
- Both Silverman and Scott bandwidth rules of thumb will be considered in this function:

Let
$$ \hat{\sigma} = \frac{1}{N} (\sigma_1+\sigma_2+...+\sigma_N) $$

Then Silverman's rule of thumb was among the optimal choice(s) covered in the lecture:

$$ h = \hat{\sigma} \cdot \left( \frac{4}{{M \cdot (N+2)}} \right)^{\frac{1}{N+4}}$$

Another rule of thumb is Scott's:

$$ h = \hat{\sigma} ⋅ M^{\frac{-1}{N+4}}$$


<br><br>
```python
g(self, x)
```
Implements $g(x)$ for the cases of a standard Gaussian and Rect bump function

- Standard Gaussian
$$g(\mathbf{x}) = \frac{1}{\sqrt{(2\pi)^N}} e^{-\frac{1}{2} \mathbf{x}^T \mathbf{x}}$$

- Rectangular

$$ g(\mathbf{x}) = \begin{cases} 1 & \text{if } -\frac{1}{2} \leq x_i \leq \frac{1}{2} \:  \forall_{i} \\ 0 & \text{otherwise} \end{cases}$$

- The input assumed here is a numpy array of dimensions `(m,n)` and the output is a numpy array of dimensions `(m)` the evaluates the probability of each point.
<br><br>


```python
ϕ(self, x)
```
- Implements $\phi$ as defined in $(1)$
- Thanks to the fact that $g$ is vectorized, $ϕ(x-x_m)$ can be computed at once for all $x_m$ by passing them in a single numpy array

<br><br>
```python
P(self, x)
```
- Implements $P$ as defined in $(2)$. The sum should be performed using Numpy for efficiency.
<br><br>


```python
transform(self, x_data)
```
- Applies $P(x)$ for each row in the given `(m,n)` numpy array `x_data` to return a numpy array `(m,)` of probabilities
<br><br>


```python
fit_transform(self, x_data)
```
- Applies `fit` followed by `transform`

#### Now go to KDEstimator.py and come back for tests


## Unit Tests

These are just some basic tests. Feel free to add more.

In [ ]:
%load_ext autoreload
%autoreload 2
# No need to restart the notebook upon change thanks to autoreload

import numpy as np
from KDEstimator import KDEstimator

Test Scott and Silverman Bandwidth

In [ ]:
# this data has standard deviation 1 along each column
x_data= np.array([
    [0, 2],
    [2, 0],
    [0, 2],
    [2, 0]
])

# create two KDEstimator objects
kde1 = KDEstimator(bump='Gauss', bandwidth='Silverman')
kde2 = KDEstimator(bump='Gauss', bandwidth='Scott')

# fit the data
kde1.fit(x_data)
kde2.fit(x_data)

# TODO 1: Assume avg_σ=1; compute Silverman and Scott with calculator and assert their values
calculated_silverman = None
calculated_scott = None

assert calculated_silverman == kde1.h
assert calculated_scott == kde2.h

Test Gaussian Bump

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

mean = np.zeros(2)  
covariance = np.eye(2)  
scipy_norm = multivariate_normal.pdf(x_data, mean=mean, cov=covariance)
our_norm = kde1.g(x_data)

# Test the implementation
assert np.allclose(our_norm, scipy_norm, atol=1e-4), "PDF values do not match expected values."

Test Rectangular Bump

In [ ]:
x_val = np.array([
    [0, -0.6, 0.6],
    [0, -0.3, 0.3],
    [0.5, -0.5, 0.5],
    [0, 0.1, 0.4],
    [3, -2, 0]
])
kde1 = KDEstimator(bump='Rect', bandwidth='Silverman')
assert np.all(kde1.g(x_val) == np.array([0, 1, 1, 1, 0]))

## End-to-end Tests

### Test Gaussian Kernel with Silverman

Note that because Scipy computes the entire covariance matrix (doesn't make a standard normal assumption), we have allowed for a 1% error. This is also why we will skip testing a float bandwidth (but it should work anyway).

In [ ]:
from scipy.stats import gaussian_kde

np.random.seed(42)
x_data = np.random.rand(3000, 2)

kde = KDEstimator(bump='Gauss', bandwidth='Silverman')
kde.fit(x_data)
my_res = kde.transform(x_data)

kernel = gaussian_kde(x_data.T, bw_method='silverman')
res = np.array(kernel(x_data.T))

np.allclose(res, my_res, rtol=0.01)

### Test Gaussian Kernel with Scott

In [ ]:
np.random.seed(42)
x_data = np.random.rand(3000, 2)

kde = KDEstimator(bump='Gauss', bandwidth='Scott')
my_res = kde.fit_transform(x_data)

kernel = gaussian_kde(x_data.T, bw_method='scott')
res = np.array(kernel(x_data.T))

np.allclose(res, my_res, rtol=0.01)

#### Test Gaussian Kernel with Float Bandwidth

In [ ]:
# Should yield no error
kde = KDEstimator(bump='Gauss', bandwidth=0.5)
kde.fit(x_data)
my_res = kde.transform(x_data)

## 😎 Put on Your Machine Learning Engineer Glasses

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def plot_3d(x_data, *, densities, titles):
    # TODO 2: Find min and max of x1 and x2 (first two columns of x_data)
    x1_min, x1_max = None
    x2_min, x2_max = None
    
    # TODO 3: Generate 100 points between min and max values for each dimension
    x1 = None
    x2 = None
    
    # TODO 4: Form a meshgrid from x1 and x2
    x1, x2 = None
    
    # convert the meshgrid into a (m, n) array to allow evaluation by the model
    x_plot = np.c_[x1.ravel(), x2.ravel()]
    
    # densities is a list of fitted kde instances.
    # TODO 5: Form a list of probabilities evaluated from x_plot by each kde instance using list comprehension
    z = None
    
    # put z in meshgrid shape as the plot function assumes that
    z = [probs.reshape(x1.shape) for probs in z]
    
    # Number of subplots
    num_subplots = len(titles)

    # Creating subplots
    plt.style.use('dark_background')
    fig, axs = plt.subplots(1, num_subplots, figsize=(6 * num_subplots, 7), dpi=140, subplot_kw={'projection': '3d'})

    # for each density we want a plot
    for i, ax in enumerate(axs):
        # TODO 6: Plot z[i] with x1 and x2
        ax.plot_surface(None, None, None, cmap='plasma', alpha=0.8)
        
        # Set title and labels
        ax.set_title(titles[i])
        ax.set_xlabel('Feature X')
        ax.set_ylabel('Feature Y')
        ax.set_zlabel('Z-axis', labelpad=1)
        # turn off grid and z-axis ticks
        ax.grid(False)
        ax.set_zticks([])
        # turn off planes
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        # change view angle
        ax.view_init(elev=50, azim=130)

    # Show the plot
    plt.show()

#### 1. Compare the Choice of Bandwidth

In [ ]:
# Generate some random data
x_data, _= datasets.make_blobs(n_samples=1000, centers=3, cluster_std=4.0, center_box=(-10,10), random_state=20)

# TODO 7: Pass bandwidth of 0.1, 0.5, 1.0, 2.0, 10.9 to each following instantiation respectively

kde1 = KDEstimator(bump='Gauss')
kde1.fit(x_data)

kde2 = KDEstimator(bump='Gauss')
kde2.fit(x_data)

kde3 = KDEstimator(bump='Gauss')
kde3.fit(x_data)

kde4 = KDEstimator(bump='Gauss')
kde4.fit(x_data)

kde5 = KDEstimator(bump='Gauss')
kde5.fit(x_data)


plot_3d(x_data, densities=[kde1, kde2, kde3, kde4, kde5], titles=['BW=0.1', 'BW=0.5', 'BW=1.0', 'BW=2.0', 'BW=10.0'])

Answer the following questions:
- Why does the estimate get smoother as we increase the bandwidth and get more discontinuous for vice versa?

- Which of the following represents the most optimal choice in your opinion as someone who has seen real data densities before?

- Which of these densities integrate to 1?

#### 2. Compare Scott and Silverman

In [ ]:
# TODO 8: Pass bandwidth='Silverman' for kde1 and 'Scott' for kde2

kde1 = KDEstimator(bump='Gauss')
kde1.fit(x_data)

kde2 = KDEstimator(bump='Gauss')
kde2.fit(x_data)

plot_3d(x_data, densities=[kde1, kde2], titles=['Silverman', 'Scott'])

[Extra] Read online (e.g., [here](https://stats.stackexchange.com/questions/90656/kernel-bandwidth-scotts-vs-silvermans-rules)) to deduce why there are similar

In [ ]:
'''
Answer goes here.
'''

#### 3. Compare Bump Functions

In [ ]:
# TODO 8: Pass bump='Gauss' for kde1 and 'Rect' for kde2

kde1 = KDEstimator()
kde1.fit(x_data)

kde2 = KDEstimator()
kde2.fit(x_data)

plot_3d(x_data, densities=[kde1, kde2], titles=['Gaussian Bump', 'Rect Bump'])

Answer the following questions:
- Why is the Rect bump more discontinuous?

- What other density estimator is equivalent to using the rect bump?

In [ ]:
'''
Answer goes here
'''

#### [TODO 9] Change the bandwidth setting for the Rect bump so that the density becomes smooth while remaining as close as possible to the Gaussian version:

In [ ]:
kde1 = KDEstimator(bump='Gauss')
kde1.fit(x_data)

kde2 = KDEstimator(bump='Rect')
kde2.fit(x_data)

plot_3d(x_data, densities=[kde1, kde2], titles=['Gaussian Bump', 'Rect Bump'])

### Perfection 👌

<div align="center">
<img width="1300" src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExbGszZWVjM2RncGRhYnFhMnc5Z2h6ZmRxbzdyZTgwMHZqMDcycmplYyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/hWkg5NRbpwW9yIDV3r/giphy.gif">
</div>